# 単一責任の原則の問題点

## 単一責任の原則を遵守したコード例
*  前回示した以下のコードは，ホームシアターシステムをイメージしている
*  このコードでは，ホームシアターシステムの機能である照明操作，プロジェクタ操作，サウンドシステム操作を，`Lighting`クラス，`Projector`クラス，`SoundSystem`クラスに分けて定義している
*  よって，単一責任の原則に遵守したコードになる
*  詳細は[第8回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook08.ipynb)を参照

> <img src="./fig/SOLID_SRP_example4b.jpg" width="350">


In [ ]:
class Lighting:
    def on(self) -> None:
        print('照明をオンにします')

    def off(self) -> None:
        print('照明をオフにします')

class Projector:
    def on(self) -> None:
        print('プロジェクタをオンにします')

    def off(self) -> None:
        print('プロジェクタをオフにします')

class SoundSystem:
    def on(self) -> None:
        print('サウンドシステムをオンにします')

    def off(self) -> None:
        print('サウンドシステムをオフにします')

# システムの操作
lighting: Lighting = Lighting()
projector: Projector = Projector()
sound_system: SoundSystem = SoundSystem()

print('--- ホームシアターシステムを起動します ---')
lighting.on()
projector.on()
sound_system.on()
print('--- 映画鑑賞中 ---')
print('--- ホームシアターシステムを終了します ---')
sound_system.off()
projector.off()
lighting.off()

## 単一責任の原則の問題点
*  上の例のように，クラスを細かく分けて定義することで，単一責任の原則を遵守できる
*  しかし，クラスを細かく分けて定義することで，**クラスの利用が複雑になる**
*  上の例の場合，それぞれのクラスからインスタンスを生成し，メソッドを順番に呼び出す必要があるので，記述する順番を間違える可能性がある
*  また，システムの手続きに変更があった場合に，修正する手間がかかるかもしれない
*  この問題を解決するための方法に，デザインパターンの一種である Facade（ファサード）パターンと呼ばれるパターンを用いる方法がある

# Facadeパターン
*  Facadeパターンは，複雑なサブシステム（クラス）に対して簡単なインターフェースを提供する
*  つまり，複雑な処理をまとめてシンプルに見せるためのパターン
*  上の例でいうと，`Lighting`クラス，`Projector`クラス，`SoundSystem`クラスがサブシステム
*  クライアントは複数のクラスや複雑な操作を直接扱うのではなく，シンプルで統一されたインターフェースを通じてシステムを利用できる
*  ここで，クライアントとは，クラスを利用するもの，すなわち，クラスのインスタンスからメソッドを呼び出すものを指す
*  上の例でいうと，ホームシアターシステムの利用者（23行目以降の処理）がクライアントに対応する

## 一般的なパターン
* Facadeパターンのクラス図は下図のように描ける

> <img src="./fig/Facade_templete.jpg" width="450">

### `Facade`クラス
*  `Facade`クラスは，Facadeパターンにおいて中心的な役割を果たすクラス
*  サブシステム群に対して「簡易的な統合インターフェース」を提供する
*  `Facade`クラスは，複雑なサブシステムの利用に対する「窓口」として機能し，クライアントがシステムを簡単に利用できるようにする役割を果たす
*  属性として，サブシステムのインスタンスを保持する（上図の`a`, `b`, `c`に対応）
*  クライアントに分かりやすいメソッド（上図の`operation`に対応）を提供する
*  メソッドの内部では，サブシステムの複数メソッド（上図の`operationA`, `operationB`, `operationC`に対応）を組み合わせて一括で呼び出す

### サブシステム群（下位モジュール群）
* `Facade`クラスが属性として所有するサブシステム（部品）の集まりのこと（上図の`SubsystemA`, `SubsystemB`, `SubsystemC`に対応）
* 上図ではサブシステム群を3つとしているが，任意の数のサブシステム群を定義できる
* 各サブシステムが一つのクラスに対応し，細かな責務をそれぞれが持つ
* 基本的には，`Facade`クラスとサブシステムの関連はコンポジションになる
* ただし，状況（例えば，テストのしやすさを重視する場合など）によっては集約を用いることもある（下図）
>* 集約にすることで，実際のオブジェクトでなく，テスト用のオブジェクト（モックと呼ばれる）を使ってテストできる
* サブシステムのメソッドは，`Facade`クラスを介して呼び出される
* サブシステムは`Facade`の中身を知らないくてよい ⇒ 結びつき（依存関係）は一方向

> <img src="./fig/Facade_templete_agg.jpg" width="460">

### クライアント
* 上図のシステムの利用者のこと
* サブシステムの複雑さを知らなくても，`Facade`経由で利用できる
* つまり，`Facade`クラスのインスタンスだけを用いればよい

## Facadeパターンのメリット・デメリット

| 観点          | メリット                         | デメリット                                     |
| ----------- | ---------------------------- | -------------------------------------- |
| **使いやすさ**   | 複雑なサブシステムをシンプルに利用できる         | 提供される機能が限定的で，細かい機能にアクセスしづらい            |
| **保守性**     | サブシステムの変更をクライアントに影響させずに隠蔽できる | `Facade`に機能を詰め込みすぎると「God Object」化して逆に複雑化 |
| **拡張性**     | クライアントコードは修正なしでサブシステム内部を拡張可能 | 新機能を使うには `Facade` にも新しいメソッドを追加する必要がある    |
| **理解のしやすさ** | クライアントにとって「システムの窓口」が一目で分かる   | クライアントが「`Facade`＝システム全体」と誤解する可能性         |



## Facadeパターンの例 その1
*  上述したホームシアターシステムの例に対して，Facadeパターンを適用する
*  Facadeパターンを使うと，単一責任の原則の問題を回避でき，以下の効果が期待できる
>*  クライアント側のコードの記述量が少なくなる
>*  メソッドの呼び出し順を間違えなくなる
>*  呼び出し順などの手続きが変わっても，Facadeクラスの中身を変えるだけでよい
*  クラス図は下図のように描ける

> <img src="./fig/HomeTheaterFacade.jpg" width="550">

In [ ]:
# SubsystemA
class Lighting:
    def on(self) -> None:
        print('照明をオンにします')

    def off(self) -> None:
        print('照明をオフにします')

# SubsystemB
class Projector:
    def on(self) -> None:
        print('プロジェクタをオンにします')

    def off(self) -> None:
        print('プロジェクタをオフにします')

# SubsystemC
class SoundSystem:
    def on(self) -> None:
        print('サウンドシステムをオンにします')

    def off(self) -> None:
        print('サウンドシステムをオフにします')

# Facadeクラスの定義
class HomeTheaterFacade:
    def __init__(self) -> None:
        self.lighting: Lighting = Lighting()
        self.projector: Projector = Projector()
        self.sound_system: SoundSystem = SoundSystem()

    def startup(self) -> None:
        print('--- ホームシアターシステムを起動します ---')
        self.lighting.on()
        self.projector.on()
        self.sound_system.on()

    def shutdown(self) -> None:
        print('--- ホームシアターシステムを終了します ---')
        self.sound_system.off()
        self.projector.off()
        self.lighting.off()
        
# クライアント
# Facadeクラスのインスタンス生成
home_theater: HomeTheaterFacade = HomeTheaterFacade()
home_theater.startup() # システム起動
print('--- 映画鑑賞中 ---')
home_theater.shutdown() # システム終了

### 集約を用いた場合
* 集約を用いると，クライアント側でサブシステムのインスタンスを生成する処理が追加される

> <img src="./fig/HomeTheaterFacade_agg.jpg" width="550">

In [ ]:
# SubsystemA
class Lighting:
    def on(self) -> None:
        print('照明をオンにします')

    def off(self) -> None:
        print('照明をオフにします')

# SubsystemB
class Projector:
    def on(self) -> None:
        print('プロジェクタをオンにします')

    def off(self) -> None:
        print('プロジェクタをオフにします')

# SubsystemC
class SoundSystem:
    def on(self) -> None:
        print('サウンドシステムをオンにします')

    def off(self) -> None:
        print('サウンドシステムをオフにします')

# Facadeクラスの定義
class HomeTheaterFacade:
    def __init__(self, lighting: Lighting, projector: Projector, sound_system: SoundSystem) -> None:
        self.lighting = lighting
        self.projector = projector
        self.sound_system = sound_system

    def startup(self) -> None:
        print('--- ホームシアターシステムを起動します ---')
        self.lighting.on()
        self.projector.on()
        self.sound_system.on()

    def shutdown(self) -> None:
        print('--- ホームシアターシステムを終了します ---')
        self.sound_system.off()
        self.projector.off()
        self.lighting.off()
        
# クライアント
# サブシステムのインスタンス生成
lighting: Lighting = Lighting()
projector: Projector = Projector()
sound_system: SoundSystem = SoundSystem()
# Facadeクラスのインスタンス生成
home_theater: HomeTheaterFacade = HomeTheaterFacade(lighting, projector, sound_system)
home_theater.startup() # システム起動
print('--- 映画鑑賞中 ---')
home_theater.shutdown() # システム終了

## Facadeパターンの例 その2
*  下図のようなECサイトで利用されるシステムをイメージしたものを実装する
*  3つのサブシステム
>* `PaymentSystem`: 決済処理を行う`pay`メソッドを持つ
>* `InventorySystem`: 在庫更新を行う`update`メソッドを持つ
>* `ShippingSystem`: 配送手配を行う`ship`メソッドを持つ
* `Facade`クラス
>* 3つのサブシステムをまとめる窓口
>* `place_order`メソッドの内部で，サブシステムの責務である「決済処理 ⇒ 在庫更新 ⇒ 配送手配」に対応するメソッドを順番に呼び出す
* これにより，クライアントは細かいサブシステムを意識せずに一つの操作（注文処理） として利用できる
* クライアントは `OrderFacade` だけ知っていればOK
* クライアント側の操作
>* `OrderFacade`クラスのインスタンスを生成 ⇒ `order`に代入
* `order.place_order`を呼び出す ⇒ 裏で複雑な処理が順番に実行される

> <img src="./fig/OrderFacade.jpg" width="450">

In [ ]:
# SubsystemA
class PaymentSystem:
    def pay(self) -> None:
        print('決済処理を実行しました')

# SubsystemB
class InventorySystem:
    def update(self) -> None:
        print('在庫を更新しました')

# SubsystemC
class ShippingSystem:
    def ship(self) -> None:
        print('配送を手配しました')

# Facadeクラス
class OrderFacade:
    def __init__(self) -> None:
        self.payment: PaymentSystem = PaymentSystem()
        self.inventory: InventorySystem = InventorySystem()
        self.shipping: ShippingSystem = ShippingSystem()

    def place_order(self) -> None: # 注文処理をまとめて実行
        self.payment.pay()
        self.inventory.update()
        self.shipping.ship()


# クライアント
order: OrderFacade = OrderFacade()
order.place_order()

# Command パターン

## Command（コマンド）パターンとは
* デザインパターンの一つで，「操作（処理要求）」をオブジェクトとして表現するしくみ
* 処理を「呼び出す側」と「実際に行う側」に分離できる
* 「やること（命令）」をクラスに包んで使い回す発想

## 一般的なパターン
* Commandパターンのクラス図は下図のように描ける

> <img src="./fig/Command_templete.jpg" width="650">

### Command（コマンド）
* 処理を抽象化したインターフェース
* 共通のインターフェース（上図の`execute`に対応）を定義する
* 「処理を実行せよ」という抽象的な契約を示す

### ConcreteCommand（具象コマンド）
* `Command` を実装したクラス
* 実際の処理を定義し，`Receiver` に処理を委譲する

### Receiver（受け手）
* 実際の処理（ビジネスロジック）を持つ
* 例：電気をつける / ファイル保存する / メッセージ送る など
* `Receiver` を使わず，`Command` に実際の処理を埋め込むと，責務混在，再利用性/テスト性の低下が起きやすい
* `Receiver` と `Command` に分けることで，変更に強いシステムの構造になる

### Invoker（呼び出し側）
* `Command`を所有し，必要なときに処理（上図の`invoke`に対応）を呼び出す
* `Command`を所有（登録）する際のメソッドを定義する（上図の`set_command`に対応）
* `Invoker`は，処理の詳細を知らなくてもよい

### Client（クライアント）
* システム利用者を表すクラスとした
* `Command`と`Receiver`を組み合わせて，`Invoker`に登録する

## Commandパターンの直感的イメージ

### 例：リモコンの操作に例えると...
* **Receiver** = テレビ本体
* **Command** = 「電源オン」「チャンネル変更」などの操作をオブジェクト化
* **Invoker** = リモコン（ボタンを押すとコマンドを実行する）
* **Client** = ユーザが「どのボタンにどの操作を割り当てるか」を決める

### 簡単なコード例
* 以下のコードは，一般的なCommandパターンのクラス図に対応する最小限のコードになっている
* `if`文の条件式に使用している`self.__command`は，属性`__command`にコマンド（`Command`）が格納されていれば`True`，そうでなければ（`None`のとき）`False`となる
* このコードでは，`ConcreteCommand` が `Receiver` を所有している，すなわち，集約の関連があるように見えるが，「全体-部分の関係」というよりは，単なる「依存関係」といえるので，クラス図では破線の矢印で結んでいる

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Commandインターフェース
class Command(ABC):
    @abstractmethod
    def execute(self) -> None:
        pass

# Receiver（実際の処理を持つ役割）
class Receiver:
    def action(self) -> None:
        print('Receiver: 処理を実行しました')

# ConcreteCommand（具体的な命令）
class ConcreteCommand(Command):
    def __init__(self, receiver: Receiver) -> None:
        self.__receiver = receiver

    def execute(self) -> None:
        self.__receiver.action() # 実際の処理をReceiverに委譲

# Invoker（呼び出し役）
class Invoker:
    def __init__(self) -> None:
        self.__command: Optional[Command] = None

    def set_command(self, command: Command) -> None:
        self.__command = command

    def invoke(self) -> None:
        if self.__command:
            self.__command.execute()
        else:
            print('Invoker: コマンドが設定されていません')


# Client（クライアント）
receiver: Receiver = Receiver()
command: ConcreteCommand = ConcreteCommand(receiver)
invoker: Invoker = Invoker()

invoker.set_command(command)
invoker.invoke()

## Commandパターンのメリット・デメリット

| 観点       | メリット                                       | デメリット                                          |
| -------- | ------------------------------------------ | ---------------------------------------------- |
| **拡張性**  | 新しいコマンドをクラスとして追加するだけで拡張できる（後述するOCPを満たす）           | クラスの数が増えやすく，シンプルなケースでは冗長に見える                   |
| **結びつき**  | 実行役（Invoker）は処理の中身を知らなくてよい ⇒ クラス間の依存が減る | 仕組みを理解するのに学習コストがかかる                             |
| **再利用性** | コマンドをオブジェクト化することで再利用可能       | 小さな処理には回りくどく見える                     |
| **柔軟性**  | Undo/Redo（元に戻す/やり直し），操作ログ再生などを自然に実装できる          | Receiver が多いと関連が複雑になる                    |
| **テストのしやすさ** | CCommand単位でモック化してテスト可能    | Command と Receiver の依存関係管理が増え，テスト準備が大変になることもある |
| **責務分離** | Invoker／Command／Receiver がそれぞれ 1 つの責務だけを持つ（SRPを満たす）    | 細かく分けすぎると逆にわかりにくくなる |



## Commandパターンの例
* ここでは，単一責任の原則に違反しているコードを，Commandパターンを使って原則に遵守したコードに改善する例を考える

### 単一責任の原則に違反している例
*  以下のコードは，タスクを文字列コマンドで実行する`TaskRunner`クラス（下図）を実装している

> <img src="./fig/Command_example1a.jpg" width="250">

* `TaskRunner`は，以下の3つの責務を同時に担っている
>* **入力の解釈**: `if`文による条件分岐で，受け取った文字列がどの処理（`backup`, `report`, `notify`）かを解釈する
>* **実際の処理**: `print('バックアップを実行しました')`などのように，実際の処理を行う
>* **ログの記録**: `self.logs.append('backup done')`などのように，実行結果を`logs`に記録する
* `TaskRunner`は，シンプルなタスク実行クラスではあるが，複数の責務が集中しているので単一責任の原則に違反しているので，以下のような問題がある
* このままだと，新しいコマンドが追加されるたびに，`if`文を修正する必要がある ⇒ 保守性が低い

In [ ]:
class TaskRunner:
    def __init__(self) -> None:
        self.logs: list[str] = []

    def run(self, command: str) -> None:
        # 入力の解釈（責務1）
        if command == 'backup':
            # 実際の処理（責務2）
            print('バックアップを実行しました')
            # ログ（責務3）
            self.logs.append('backup done')
        elif command == 'report':
            print('レポートを作成しました')
            self.logs.append('report done')
        elif command == 'notify':
            print('通知を送信しました')
            self.logs.append('notify done')
        else:
            print('未知のコマンドです')
            self.logs.append('unknown command')

### 単一責任の原則を遵守したコードに改善
* ここで，Commandパターンを使って単一責任の原則を遵守したコードに改善する
* 改善したコードは以下のとおりで，クラス図は下図のように描ける
* `Command`クラス:
>* 処理（コマンド）の実行に対するインターフェースを抽象メソッド `execute` として定義
* `Ops`クラス:
>* `Ops`は「Operations」の略
>* `Receiver`の役割を持つクラスで，3つの処理（バックアップの実行，レポート作成，通知送信）の実体を持つ
>* `run_backup`メソッド: バックアップの実行
>* `generate_report`メソッド: レポート作成
>* `send_notification`メソッド: 通知送信
* `Logger`クラス:
>* 処理の記録だけに責務を限定
>* インスタンス属性の `logs: list[str]` にログを蓄積していく（本来はプライベート属性のほうが望ましい）
>* 記録方式に変更があっても，コマンド側の変更を最小化できる
>* `Logger`は記録を専任するためのクラスで，`Command`パターンの構成要素ではないが，単一責任の原則を遵守するために定義した
* `BackupCommand` / `ReportCommand` / `NotifyCommand`:
>* `ConcreteCommand`の役割を持つクラス
>* 2つのインスタンス属性 `__ops: Ops` と `__logger: Logger` を持つ
>* `execute` を実装し，1つの処理に責務を限定
>* `execute`の実装内容: 
>>* 対応する `Ops` のメソッド（`run_backup` or `generate_report` or `send_notification`）を呼び出して処理を実行
>>* `Logger.add`を呼び出して結果を記録する
* `Invoker`クラス
>* 命令を呼び出す役割のクラス
>* 入力（コマンド名）に応じて，対応する処理を実行する（`execute`を呼び出す）
>* インスタンス属性`__commands: dict[str, Command]`で，コマンド名と対応する処理（`execute`）を実装するインスタンスを保持
>* コマンド名と対応する処理は，`register`メソッドで登録
>* `handle`メソッドで，コマンド名を引数として取得し，対応するインスタンスから `execute` を呼び出す
>* `Invoker` は処理（コマンド）の中身を知らないので，コマンドの追加・変更があっても影響を受けない

> <img src="./fig/Command_example1b.jpg" width="850">

* このコードは，以下のように責務が分担されているので，単一責任の原則に遵守しているといえる
>* `Ops`: 実際の処理を実行するだけ
>* `Logger`: 記録するだけ
>* `ConcreteCommand`: 特定の処理（メソッド）をまとめて呼び出すだけ
>* `Invoker`: 入力（コマンド名）応じて `execute` を呼び出すだけ

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Commandインターフェース
class Command(ABC):
    @abstractmethod
    def execute(self) -> None:
        pass

# Receiver（実処理の中身）
class Ops:
    def run_backup(self) -> None:
        print('バックアップを実行しました')

    def generate_report(self) -> None:
        print('レポートを作成しました')

    def send_notification(self) -> None:
        print('通知を送信しました')

# Logger（記録専任）
class Logger:
    def __init__(self) -> None:
        self.__logs: list[str] = []
  
    def add(self, message: str) -> None:
        self.__logs.append(message)

    def show(self) -> None:
        print(self.__logs)

# ConcreteCommands（バックアップ実行コマンド）
class BackupCommand(Command):
    def __init__(self, ops: Ops, logger: Logger) -> None:
        self.__ops: Ops = ops
        self.__logger: Logger = logger

    def execute(self) -> None:
        self.__ops.run_backup()
        self.__logger.add('backup done')

# ConcreteCommands（レポート作成コマンド）
class ReportCommand(Command):
    def __init__(self, ops: Ops, logger: Logger) -> None:
        self.__ops: Ops = ops
        self.__logger: Logger = logger

    def execute(self) -> None:
        self.__ops.generate_report()
        self.__logger.add('report done')

# ConcreteCommands（通知送信コマンド）
class NotifyCommand(Command):
    def __init__(self, ops: Ops, logger: Logger) -> None:
        self.__ops: Ops = ops
        self.__logger: Logger = logger

    def execute(self) -> None:
        self.__ops.send_notification()
        self.__logger.add('notify done')


# Invoker（命令を呼び出す役割）
class Invoker:
    def __init__(self) -> None:
        self.__commands: dict[str, Command] = {}

    def register(self, name: str, command: Command) -> None:
        self.__commands[name] = command

    def handle(self, name: str) -> None:
        cmd: Optional[Command] = self.__commands.get(name)
        if cmd is None:
            print('未知のコマンドです')
            return
        cmd.execute()

# クライアント
ops: Ops = Ops()
logger: Logger = Logger()

invoker: Invoker = Invoker()
invoker.register('backup', BackupCommand(ops, logger))
invoker.register('report', ReportCommand(ops, logger))
invoker.register('notify', NotifyCommand(ops, logger))

# 入力に応じて実行
invoker.handle('backup')
invoker.handle('report')
invoker.handle('unknown')
invoker.handle('notify')

# ログの表示
print(logger.logs)


# 開放・閉鎖の原則（OCP: Open-Closed Principle）
*  情報システム（ソフトウェア）の構成要素（クラス，モジュール，関数など）は，拡張に対して開かれており，一方，修正・変更に対しては閉じているべきであるという原則
*  つまり，既存のコードを変更せずに，新機能を追加できるような設計を目指すべきということ
*  開放（Open）: 新しい機能や振る舞いを追加するために，既存のコードを拡張できる状態
*  閉鎖（Closed）: 既存のコードを変更する必要がない状態
*  この原則に従うことで，バグの発生リスクを低減し，コードの安定性を保つことができる

## 開放・閉鎖の原則を遵守するための方法
*  開放・閉鎖の原則を遵守するためには，主に抽象化とポリモーフィズムを活用する
*  具体的には，抽象クラスやインターフェースを定義し，実装をサブクラスや実装クラスに委譲する
*  これにより，新しい機能（クラス）を追加する際に，既存のクラスを変更せずに拡張できる

## 簡単な例: 動物の鳴き声を出すシステム

### SRPとOCPに違反するコード
* クラス図は以下のとおり

> <img src="./fig/SRP-OCP_sample1a.jpg" width="280">

* このコードは，SRPとOCPに違反している
* SRPに違反している理由: `Animal`クラスは，犬の鳴き声を出す役割と猫の鳴き声を出す役割がある
* OCPに違反している理由: 新しい動物（例: `Cow`など）を追加するたびに，`Animal`クラスを変更しなければならない ⇒ その結果，`Animal`クラスが複雑化する

In [ ]:
class Animal:
    
    def make_sound(self, animal_type: str) -> None:
        if animal_type == 'Dog':
            print('ワンワン')
        elif animal_type == 'Cat':
            print('ニャー')

# クライアント側
animal = Animal()
animal.make_sound('Dog')
animal.make_sound('Cat') 

### SRPに従ったコードに修正
*  2つの役割を持っていた`Animal`クラスを2つのクラス`Dog`と`Cat`に分割することで，SRPに従ったコードに修正できる
*  ここで，鳴き声を出す機能に対するクライアント（利用者）として，`sound_producer`関数を定義した

> <img src="./fig/SRP-OCP_sample1b.jpg" width="400">

*  しかし，このコードには以下の問題が残されている
>*  既存のクラスに影響を与えずに新しい動物クラス（`Cow`クラスなど）を定義できるが，`make_sound`メソッドを持たないクラスが定義できてしまう
>*  新しい動物クラスを追加すると，`sound_producer`関数における仮引数`animal`に対する型ヒントの記述を変更する必要がある
*  したがって，このコードはOCPに従ったコードとしては十分ではないといえる

In [ ]:
from typing import Union

class Dog:
    
    def make_sound(self) -> None:
        print('ワンワン')

class Cat:
    
    def make_sound(self) -> None:
        print('ニャー')

# クライアント
def sound_producer(animal: Union[Dog, Cat]) -> None:
    animal.make_sound()

dog = Dog()
cat = Cat()
sound_producer(dog)
sound_producer(cat)

### OCPに従ったコードに修正
*  OCPに従った，さらによいコードにするために，抽象化とポリモーフィズムを使う
*  `Animal`クラスを抽象クラス（インターフェース）にして，`make_sound`メソッドを抽象メソッドとして定義
*  具体的な動物クラス（`Dog`や`Cat`）は，`Animal`クラスを継承するサブクラスとし，各クラスで`make_sound`メソッドの実装を定義する
*  これにより，新しい動物クラスを追加する際に`make_sound`メソッドの定義を強制できる（共通のインターフェースが定義できる）
*  また，クライアントの`sound_producer`関数が具象クラスに依存しなくなるので，動物クラス（具象クラス）に修正や変更があっても，クライアントを変更する必要がない

> <img src="./fig/SRP-OCP_sample1c.jpg" width="520">

*  この例のように，抽象クラス（インターフェース）を使って共通のインターフェースを定義し，クライアントは異なる実装を定義している具象クラスを切り替えて利用するといったコードの記述方法は，**Strategyパターン**と呼ばれている
*  Strategyパターンは次回説明する（[第10回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook10.ipynb)を参照）

In [ ]:
from abc import ABC, abstractmethod

class Animal(ABC):
    
    @abstractmethod
    def make_sound(self) -> None:
        pass

class Dog(Animal):
    
    def make_sound(self) -> None:
        print('ワンワン')

class Cat(Animal):
    
    def make_sound(self) -> None:
        print('ニャー')

# 新しい動物を追加する場合
class Cow(Animal):
    
    def make_sound(self) -> None:
        print('モーモー')

# クライアント
def sound_producer(animal: Animal) -> None:
    animal.make_sound()

animals = [Dog(), Cat(), Cow()] # リストを使って，まとめてインスタンス生成
for animal in animals:
    sound_producer(animal)

## OCPを遵守することによる効果

### メリット
*  拡張性の向上: 新しい機能を追加する際に，既存のコードの変更が不要になる ⇒ システムを容易に拡張できる
*  保守性の向上: 既存のコードに手を加えることなく新機能を追加できる ⇒ バグのリスクを減らし，保守作業が簡単になる
*  再利用性の向上: 汎用的な部品を作成しやすくなる ⇒ コードの再利用が容易になる
*  可読性の向上: コードが適切に分割されているので理解しやすい
*  試験性の向上: 既存のコードに影響を与えずに新しい部品を追加できる ⇒ 新しい部品だけテストすればよい

### デメリット
*  設計の複雑化: 抽象化やインターフェースの導入により，初期の設計が複雑になる場合がある
*  過剰な抽象化: 必要以上に抽象化を行うと，コードの理解や追跡が難しくなることがある

## 具体例: 図形の面積を計算するシステム

### OCPに違反するコード
*  以下のコードにおける`Rctangle`クラスは，一つの矩形（四角形）を表現しているクラスで，インスタンス属性として矩形の幅（`width`）と高さ（`height`）を持つ
*  また，`AreaCalculator`クラスは，図形に関する演算を扱うためのクラスで，`calculate_total_area`メソッドが定義されている
*  `calculate_total_area`メソッドは，図形オブジェクト（`Rctangle`クラスのインスタンス）を要素とするリストを引数として渡すことで，それらの面積の合計を戻り値として返す


> <img src="./fig/SOLID_OCP_sample1.jpg" width="800">

In [ ]:
class Rectangle:
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

class AreaCalculator:
    def calculate_total_area(self, shapes: list[Rectangle]) -> float:
        total_area = 0.0
        for shape in shapes:
            total_area += shape.width * shape.height
        return total_area

shape_list = [Rectangle(4.0, 5.0), Rectangle(8.0, 2.0)]
calculator = AreaCalculator()
result = calculator.calculate_total_area(shape_list)
print(f'面積の合計: {result}')

### 問題点
*  上のコードにおいて，新しい図形クラス（例えば，`Triangle`クラスなど）を新たに追加すると，`AreaCalculator`クラスの内容（`calculate_total_area`メソッドの処理）を変更する必要がある
*  次のコードは，上のコードに`Triangle`クラスを追加したコード例である
*  `isinstance`関数（[第7回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook07.ipynb)参照）を使って，面積の計算方法を切り替えている
*  このコードにおいて，さらに図形クラス（例えば，`Circle`クラスなど）を新たに追加すると，再度，`AreaCalculator`クラスの内容（`calculate_total_area`メソッドの処理）を変更する必要がある

<img src="./fig/SOLID_OCP_sample2.jpg" width="800">

In [ ]:
from typing import Union

class Rectangle:
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

class Triangle:
    def __init__(self, base: float, height: float) -> None:
        self.base = base
        self.height = height

class AreaCalculator:
    def calculate_total_area(self, shapes: list[Union[Rectangle, Triangle]]) -> float:
        total_area = 0.0
        for shape in shapes:
            if isinstance(shape, Rectangle):
                total_area += shape.width * shape.height
            if isinstance(shape, Triangle):
                total_area += shape.base * shape.height / 2
        return total_area

shape_list = [Rectangle(4.0, 5.0), Triangle(3.0, 5.0)]
calculator = AreaCalculator()
result = calculator.calculate_total_area(shape_list)
print(f'面積の合計: {result}')

### OCPに従ったコードに修正
*  各図形クラスが自身の面積を計算する責任（メソッド）を持つようにする
*  具体的には，抽象クラス`shape`を定義し，その中で抽象メソッド`calculate_area`を定義する
*  具体的な図形クラス（`Rectangle`クラスや`Triangle`クラス）は`shape`クラスを継承したクラスとし，各クラスにおいて`calculate_area`メソッドの実装を定義する
  
<img src="./fig/SOLID_OCP_sample3.jpg" width="1000">


In [ ]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def calculate_area(self) -> float:
        pass

class Rectangle(Shape):
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

    def calculate_area(self) -> float:
        return self.width * self.height

class Triangle(Shape):
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

    def calculate_area(self) -> float:
        return self.width * self.height / 2

# 新しい図形を追加する場合
# class Circle(Shape):
#     def __init__(self, radius: float) -> None:
#         self.radius = radius

#     def calculate_area(self) -> float:
#         return self.radius ** 2 * 3.14

class AreaCalculator:
    def calculate_total_area(self, shapes: list[Shape]) -> float:
        total_area = 0
        for shape in shapes:
            total_area += shape.calculate_area() # 各図形オブジェクトからメソッドを呼び出す
        return total_area

shape_list = [Rectangle(4, 5), Triangle(8, 2)]
# shape_list = [Rectangle(4, 5), Triangle(8, 2), Circle(5)]
calculator = AreaCalculator()
result = calculator.calculate_total_area(shape_list)
print(f'面積の合計: {result}')

# 実習
以下の「修正前コード」は，「問題点」で指摘しているように開放・閉鎖の原則（OCP）に違反している．このコードを「コード修正要件・補足説明」に従って，OCP遵守のコードに修正しなさい．ただし，コードには型ヒントをつけ，既に入力されているコードは変更・削除しないこと．

---
**修正前コード：**
```Python
shipping_method = ['standard', 'express']

def calc_shipping_fee(method: list[str], weight_kg: float) -> int:
    if method == 'standard':
        return int(500 + 20 * weight_kg)
    elif method == 'express':
        return int(900 + 30 * weight_kg)
    else:
        raise ValueError('未知の配送方法です')
```
---
**問題点:**
* `calc_shipping_fee`は，配送方法（現時点では standard と express の2種類）に応じて送料を計算する関数であるが，新しい配送方法が増えるたびに `if`文を編集する必要がある
* よって，`calc_shipping_fee`関数は，OCPに違反している
---
**コード修正要件・補足説明:**
*  以下のクラス図と対応させる
* `ShippingPolicy`を 抽象メソッド`calc`のみを持つインターフェースとして定義する
* インターフェースを実装するための2つのクラス `StandardPolicy` と `ExpressPolicy` を定義する
* `calc` の引数 `weight_kg` は荷物の重さ（float）に対応する
* `StandardPolicy`における `calc`メソッドは `int(500 + 20 * weight_kg)`を戻り値として返る
* `StandardPolicy`における `calc`メソッドは `int(900 + 30 * weight_kg)`を戻り値として返る
---
**クラス図:**

<img src='./fig/OCP_exercise.jpg' width='430'>

---
**期待される実行結果:**
```
通常配送料金: 540円
即日配送料金: 960円
```

In [ ]:
# ここにコードを記述（必要に応じて改行する）

# --------------------------
# 動作確認（クライアント側）
# --------------------------
std: ShippingPolicy = StandardPolicy()
exp: ShippingPolicy = ExpressPolicy()
print(f'通常配送料金: {std.calc(2.0)}円')
print(f'即日配送料金: {exp.calc(2.0)}円')

# 参考資料
*  Robert C.Martin (著), 角征典 (翻訳), 高木正弘 (翻訳), [Clean Architecture 達人に学ぶソフトウェアの構造と設計](https://www.kadokawa.co.jp/product/301806000678/), KADOKAWA, 2018.
*  ひらまつしょうたろう, [Python で身につける オブジェクト指向【SOLID原則+デザインパターンで、オブジェクト指向設計 の基礎を習得！】](https://www.udemy.com/course/python-solid-design-pattern/?couponCode=KEEPLEARNING), Udemy, 最終更新日 2024/6.
*  Mark Summerfield (著), 斎藤康毅 (訳), [実践 Python 3](https://www.oreilly.co.jp/books/9784873117393/), オライリージャパン, 2015.
<!-- *  Bill Lubanovic (著), 鈴木駿 (監訳), 長尾高弘 (訳), [入門 Python 3 第2版](https://www.oreilly.co.jp/books/9784873119328/), オライリージャパン, 2021. -->
<!-- *  Guido van Rossum (著), 鴨澤眞夫 (翻訳), [Pythonチュートリアル 第4版](https://www.oreilly.co.jp/books/9784873119359/), オライリージャパン, 2021. -->

